# Estimation of a Markov State Model

In [17]:
from msmhelper.utils import datasets
traj = datasets.hummer15_4state(0.2, 0.1, nsteps=int(1e5))

In [18]:
import msmhelper as mh

In [19]:
lagtime = 1
tmat, states = mh.msm.estimate_markov_model(traj, lagtime)
tmat

array([[0.80382329, 0.19617671, 0.        , 0.        ],
       [0.19999208, 0.70056654, 0.09944139, 0.        ],
       [0.        , 0.10133635, 0.69918043, 0.19948322],
       [0.        , 0.        , 0.20373577, 0.79626423]])